<a href="https://colab.research.google.com/github/ariakhademi/telematics/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install opendatasets

In [35]:
import opendatasets as od
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim
from sklearn.preprocessing import StandardScaler

In [5]:
# data loading
dataset_url = 'https://www.kaggle.com/datasets/yunlevin/levin-vehicle-telematics'
od.download(dataset_url)
file_path = 'levin-vehicle-telematics/allcars.csv'
df = pd.read_csv(file_path)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: ariakhademi@gmail.com
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/yunlevin/levin-vehicle-telematics


100%|██████████| 587M/587M [00:10<00:00, 58.2MB/s]


<ipython-input-5-371d740e4fd4>:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


**Detect hard brakes with a threshold**

In [6]:
# constants
acceleration_threshold = 0.5

# data preprocessing
df['timeStamp'] = pd.to_datetime(df['timeStamp'])
delta_speed = df['speed'].diff()
delta_time = df['timeStamp'].diff().dt.total_seconds()
acceleration = delta_speed / delta_time
df['acceleration'] = acceleration

# detect hard brakes
df['hard_brakes'] = (df['acceleration'] < -acceleration_threshold).astype(int)
df['hard_brakes']

,hard_brakes
0,0
1,0
2,0
3,0
4,0
...,...
7214690,0
7214691,0
7214692,0
7214693,0


**Detect hard brakes with machine learning**

In [36]:
# data preprocessing
df = df[:10000]
df['timeStamp'] = pd.to_datetime(df['timeStamp'])
delta_speed = df['speed'].diff()
delta_time = df['timeStamp'].diff().dt.total_seconds()
acceleration = delta_speed / delta_time
df['acceleration'] = acceleration
scaler = StandardScaler()
df[['speed', 'acceleration']] = scaler.fit_transform(df[['speed', 'acceleration']])

# create sequences
seq_length = 10
input_sequence = []
for i in range(len(df) - seq_length):
    input_sequence.append(df[['speed', 'acceleration']].iloc[i:i+seq_length].values)
# sequence_input, e.g. = [[[1,2],[3,4],...,[x,y]],[[6,7],[8,9],...,[a,b]],...]
input_sequence = np.array(input_sequence)
input_tensor = torch.tensor(input_sequence) # convert to tensor
dataset = torch.utils.data.TensorDataset(input_tensor.float(), input_tensor.float())  # for auto encoder
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

In [37]:
# hyperparams
num_features = 2
encoded_size = 8
lr = 0.01
num_epochs = 10
input_size = seq_length * num_features
batch_size = 32

class AutoEncoder(nn.Module):
    def __init__(self,input_size,encoded_size,batch_size):
      super(AutoEncoder, self).__init__()

      # params
      self.batch_size = batch_size
      self.input_size = input_size
      self.encoded_size = encoded_size

      # encoder
      self.encoder = nn.Sequential(
          nn.Linear(input_size,encoded_size),
          nn.ReLU()
      )

      # decoder
      self.decoder = nn.Sequential(
          nn.Linear(encoded_size,input_size),
          nn.ReLU()
      )

    def forward(self, x):
      x = x.view(x.size(0), -1)
      return self.decoder(self.encoder(x))

# model, loss, optimizer
model = AutoEncoder(input_size,encoded_size,batch_size)
loss_fn = nn.MSELoss()
optim = torch.optim.Adam(params=model.parameters(), lr=lr)

In [41]:
from functools import total_ordering
torch.manual_seed(42)

# training
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    # each batch
    for input,_ in dataloader:
      optim.zero_grad()
      output = model(input)
      loss = loss_fn(output,input.view(input.size(0),-1))
      loss.backward()
      optim.step()
      # batch loss
      print(f'Epoch: {epoch+1}, Loss: {loss.item()}')
      total_loss += loss.item()

    # epoch loss
    print(f'Epoch: {epoch+1}, Loss: {loss.item()}')

Epoch: 1, Loss: 1.5978378057479858
Epoch: 1, Loss: 1.4129679203033447
Epoch: 1, Loss: 1.1399753093719482
Epoch: 1, Loss: 0.9959678649902344
Epoch: 1, Loss: 1.0024034976959229
Epoch: 1, Loss: 0.792737603187561
Epoch: 1, Loss: 0.5487877130508423
Epoch: 1, Loss: 0.715899646282196
Epoch: 1, Loss: 0.8244045376777649
Epoch: 1, Loss: 0.909244179725647
Epoch: 1, Loss: 0.5703184008598328
Epoch: 1, Loss: 0.5480901598930359
Epoch: 1, Loss: 0.6098405122756958
Epoch: 1, Loss: 0.6473111510276794
Epoch: 1, Loss: 0.6692762970924377
Epoch: 1, Loss: 0.5931586027145386
Epoch: 1, Loss: 0.8645720481872559
Epoch: 1, Loss: 0.979069709777832
Epoch: 1, Loss: 0.8080695867538452
Epoch: 1, Loss: 0.4600191116333008
Epoch: 1, Loss: 0.6043609380722046
Epoch: 1, Loss: 0.8318277597427368
Epoch: 1, Loss: 0.708304762840271
Epoch: 1, Loss: 0.7870355844497681
Epoch: 1, Loss: 0.6519812345504761
Epoch: 1, Loss: 0.5537768602371216
Epoch: 1, Loss: 0.6420091390609741
Epoch: 1, Loss: 0.6077564358711243
Epoch: 1, Loss: 0.7295933

In [ ]:
# function for hard brakes
def hard_brakes(model, input):
    with torch.inference_mode():
        model.eval()
        output = model(input)
        err = torch.mean(output - input)
        anomaly = err > acceleration_threshold